In [17]:
import os
import json
import sqlite3
import requests
from pprint import pprint
from datetime import datetime, timedelta
from typing import TypedDict, Optional, Dict, Any
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_classic.memory import ConversationBufferWindowMemory
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

In [2]:
llm = ChatOpenAI(
    temperature=0.6,
    model="glm-4.7-flash",
    openai_api_key=os.getenv("ZAI_API_KEY"),
    openai_api_base="https://api.z.ai/api/paas/v4/"
)

In [3]:
llm.invoke("Hello, how are you?").content

"Hello! I'm doing well, thank you for asking. As an AI, I don't have feelings, but I'm functioning perfectly and ready to help.\n\nHow can I assist you today?"

In [6]:
# Constants
DB_PATH = "crypto_kb.db"
STALE_MINUTES = 5  # Data older than this is considered stale
REJECTION_MESSAGE = "INSUFFICIENT DATA – Not found in Knowledge Base or API"
DISALLOWED_INTENTS = ["prediction", "advice", "hypothetical"]
API_BASE_URL = "https://api.freecryptoapi.com/v1/"
TOKEN = os.getenv("FREE_CRYPTO_API_KEY")
url = f"{API_BASE_URL}getData?symbol=BTC"

headers = {
    "Authorization": f"Bearer {TOKEN}",
}

In [10]:
response = requests.get(headers=headers, url=url)
response.status_code

200

In [22]:
print(json.dumps(response.json(), indent=3))

{
   "status": "success",
   "symbols": [
      {
         "symbol": "BTC",
         "last": "89958.11",
         "last_btc": "1",
         "lowest": "87932.07",
         "highest": "90099.5",
         "date": "2026-01-21 08:56:28",
         "daily_change_percentage": "0.28690952974195",
         "source_exchange": "binance"
      }
   ]
}
